1. Data Processing

In [2]:
import tensorflow as tf
import string
import requests

In [3]:
dataset= open('./data/Harry_Potter_all_books_preprocessed.txt', 'r',encoding='utf-8').read()

dataset[0:10000]

'THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much .They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense .Mr Dursley was the director of a firm called Grunnings which made drills .He was a big beefy man with hardly any neck although he did have a very large mustache .Mrs Dursley was thin and blonde and had nearly twice the usual amount of neck which came in very useful as she spent so much of her time craning over garden fences spying on the neighbors .The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere .The Dursleys had everything they wanted but they also had a secret and their greatest fear was that somebody would discover it .They didnt think they could bear it if anyone found out about the Potters .Mrs Potter was Mrs Dursleys sister but they hadnt met for several years in 

In [4]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens



In [5]:
tokens=clean_text(dataset)
print(tokens[0:50])

['the', 'boy', 'who', 'lived', 'mr', 'and', 'mrs', 'dursley', 'of', 'number', 'four', 'privet', 'drive', 'were', 'proud', 'to', 'say', 'that', 'they', 'were', 'perfectly', 'normal', 'thank', 'you', 'very', 'much', 'they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense', 'mr']


In [6]:
len(tokens)

1086486

In [7]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i>500000:
    break

print(len(lines))

499951


In [8]:
lines[0]

'the boy who lived mr and mrs dursley of number four privet drive were proud to say that they were perfectly normal thank you very much they were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense mr dursley'

2. Building the LSTM Model and Preparing X & Y

In [9]:
import numpy as np
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM,  Embedding

from keras_preprocessing.sequence import pad_sequences

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [11]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]

In [12]:
X[0]

array([   1,  228,   51, 1570,   82,    2,  147,  897,    5,  680,  439,
        928,  787,   26, 2040,    3,  177,   17,   21,   26, 1339,  887,
        988,   13,   59,  155,   21,   26,    1,  138,  159,  450, 1008,
          3,   31, 2189,   11,  182,  864,  120, 2442,  164,   21,   68,
         78,  743,   22,  593, 3062,   82])

In [13]:
vocab_size=len(tokenizer.word_index)+1

In [14]:
y=to_categorical(y,num_classes=vocab_size)


In [15]:
seq_length=X.shape[1]

LSTM Model

In [16]:
model=Sequential()
model.add(Embedding(vocab_size,50,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

2024-06-05 17:05:05.968037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-05 17:05:05.968902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-05 17:05:05.969357: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            806200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 16124)             1628524   
                                                                 
Total params: 2,585,624
Trainable params: 2,585,624
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
model.fit(X,y,batch_size=256,epochs=50)

Epoch 1/50


2024-06-05 17:05:06.269672: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-06-05 17:05:06.420930: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-05 17:05:06.421517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-05 17:05:06.422382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG IN

1953/1953 [==============================] - 521s 266ms/step - loss: 6.5680 - accuracy: 0.0611
Epoch 2/50
1953/1953 [==============================] - 557s 285ms/step - loss: 5.9947 - accuracy: 0.1002
Epoch 3/50
1953/1953 [==============================] - 531s 272ms/step - loss: 5.7317 - accuracy: 0.1131
Epoch 4/50
1953/1953 [==============================] - 526s 269ms/step - loss: 5.5623 - accuracy: 0.1208
Epoch 5/50
1953/1953 [==============================] - 523s 268ms/step - loss: 5.4308 - accuracy: 0.1265
Epoch 6/50
1953/1953 [==============================] - 486s 249ms/step - loss: 5.3144 - accuracy: 0.1326
Epoch 7/50
1953/1953 [==============================] - 466s 239ms/step - loss: 5.2143 - accuracy: 0.1373
Epoch 8/50
1953/1953 [==============================] - 375s 192ms/step - loss: 5.1240 - accuracy: 0.1416
Epoch 9/50
1953/1953 [==============================] - 409s 210ms/step - loss: 5.0494 - accuracy: 0.1450
Epoch 10/50
1953/1953 [==============================] - 

In [20]:
seed_text=lines[12343]
seed_text

'the words got lost on the way to his mouth and what he said instead was who are you the giant chuckled true i havent introduced meself rubeus hagrid keeper of keys and grounds at hogwarts he held out an enormous hand and shook harrys whole arm what about that tea'

In [35]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = np.argmax(model.predict(encoded),axis=1)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if (index == y_predict).all():
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [38]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 20)


1/1 [==============================] - 0s 14ms/step


'said harry and ron and hermione were looking at him and he was looking at him his eyes were fixed'

In [42]:
seed_text=lines[2111]
print(seed_text)
generate_text_seq(model, tokenizer, seq_length, seed_text, 50)

in fact it was nearly midnight before the cat moved at all a man appeared on the corner the cat had been watching appeared so suddenly and silently youd have thought hed just popped out of the ground the cats tail twitched and its eyes narrowed nothing like this man had
1/1 [==============================] - 0s 13ms/step


'been able to get up to the castle he was going to be fed to the size of a large quantity of green light and befuddling perfume of a jewelencrusted and candles and dimitrov moran asleep he had been pushed into the air and a few of them hurtled toward'